In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 20ex !important; font-weight: bold;}
.widget-checkbox {padding-right: 300px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))


class ConfigForm(object):
    # Define widgets
    caption = widgets.HTML(value='<h3>Configure Your Action</h3>')
    button = widgets.Button(description='Submit')
    action = widgets.Dropdown(options=['Insert', 'Delete', 'Display', 'Update'], value='Insert')
    datatype = widgets.RadioButtons(options=['RawData', 'ProcessedData'], value='ProcessedData')
    collection = widgets.Text(value='')
    relationships = widgets.Textarea(value='', placeholder="List each relationship on a separate line.")
    rights = widgets.Text(value='')
    OCR = widgets.Checkbox(value=False)
    processes = widgets.Textarea(value='', placeholder="List each process on a separate line.")
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='Action:'), action], layout=flex),
        Box([Label(value='Node Type:'), datatype], layout=flex),
        Box([Label(value='collection:'), collection], layout=flex),
        Box([Label(value='relationships:'), relationships], layout=flex),
        Box([Label(value='rights:'), rights], layout=flex),
        Box([Label(value='Text from OCR:'), OCR], layout=flex),
        Box([Label(value='processes:'), processes], layout=flex),
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.action = ConfigForm.form_items[0]
        self.datatype = ConfigForm.form_items[1]
        self.collection = ConfigForm.form_items[2]
        self.relationships = ConfigForm.form_items[3]
        self.rights = ConfigForm.form_items[4]
        self.ocr = ConfigForm.form_items[5]
        self.processes = ConfigForm.form_items[6]

        # Helper method
        def hide(widget):
            widget.remove_class('visible').add_class('hidden')

        # Helper method
        def show(widgets, all_widgets):
            for widget in all_widgets:
                if widget in widgets:
                    widget.add_class('visible').remove_class('hidden')
                else:
                    widget.remove_class('visible').add_class('hidden')                

        # Modify widget visibility when the form is changed
        def change_visibility(change):
            box = self.form
            datatype_widget = box.children[1]
            collection_widget = box.children[2]
            relationships_widget = box.children[3]
            rights_widget = box.children[4]
            ocr_widget = box.children[5]
            processes_widget = box.children[6]
            all_widgets = [datatype_widget, collection_widget, relationships_widget, 
                            rights_widget, ocr_widget, processes_widget]
            raw_widgets = {
                'Insert': [datatype_widget, collection_widget, relationships_widget, 
                            rights_widget, ocr_widget],
                'Update': [datatype_widget, collection_widget, relationships_widget, 
                            rights_widget, ocr_widget],
                'Display': [datatype_widget, collection_widget],
                'Delete': [datatype_widget, collection_widget]
            }

            processed_widgets = {
                'Insert': [datatype_widget, collection_widget, processes_widget],
                'Update': [datatype_widget, collection_widget, processes_widget],
                'Display': [datatype_widget, collection_widget],
                'Delete': [datatype_widget, collection_widget]
            }
            if str(ConfigForm.datatype.value) == 'RawData':
                widget_list = raw_widgets
            else:
                widget_list = processed_widgets

            active = str(ConfigForm.action.value)
            if active == 'Insert':
                show(widget_list['Insert'], all_widgets)
            elif active == 'Delete':
                show(widget_list['Delete'], all_widgets)
            elif active == 'Update':
                show(widget_list['Update'], all_widgets)
            else:
                show(widget_list['Display'], all_widgets)
                            

        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq
        
        def handle_submit(values):
            # Save the form values in a dict
            self.values = {'action': ConfigForm.action.value}
            self.values['datatype'] = ConfigForm.datatype.value 
            self.values['collection'] = ConfigForm.collection.value.strip()
            if ConfigForm.action.value == 'Insert' or ConfigForm.action.value == 'Update':
                if ConfigForm.datatype.value == 'RawData':
                    self.values['relationships'] = split_list(ConfigForm.relationships.value) 
                    self.values['rights'] = ConfigForm.rights.value.strip()
                    self.values['OCR'] = ConfigForm.OCR.value
                else:
                    self.values['processes'] = split_list(ConfigForm.processes.value)                
            print('Configuration saved. Values will be available is the cells below.')
            print(self.values)
    
        # Initialise widgets in the container Box
        self.form = Box([self.action, self.datatype, self.collection, self.relationships, 
                         self.rights, self.ocr, self.processes],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form
        for item in box.children[2:6]:
            item.add_class('hidden')
        action_field = box.children[0].children[1]
        nodetype_field = box.children[1].children[1]
        
        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        nodetype_field.observe(change_visibility)
        action_field.observe(change_visibility)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)